# Evaluation

In [383]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [384]:
import pandas as pd
import numpy as np
from src import preprocess as prep
from src.evaluation import pick_test_segments, generate_test_segments, evaluate_segments, evaluate
from src.Video import Video
from src.Segment import Segment
from src import search
import random
import cv2

## Parameters

In [502]:
NUM_VIDEOS = 2
GRID_SIZE = 2
BINS = [int(180/4), int(256/4)]
# negative value is average; -2 averages two frames, takes every 2nd frame (only skips one) (if frame_id % 2 == 0).
HIST_FRAME_SKIP = -2
REFRESH = False

# vergeet gebruikte params soms dus print ze maar afentoe
def printParams():
    print('Num. Vid {} - Grid {} - Bins {} - Skip {}'.format(NUM_VIDEOS, GRID_SIZE, BINS, HIST_FRAME_SKIP))

## Load training set / generate test set

In [503]:
printParams()
training_set = prep.load_training_set(range(1, NUM_VIDEOS+1), GRID_SIZE,
                                      BINS, HIST_FRAME_SKIP, force_refresh=REFRESH)

Num. Vid 2 - Grid 2 - Bins [45, 64] - Skip -1


In [489]:
# Set of 100 custom fragments with duration 20sec
test_set, labels = prep.get_test_video_set(NUM_VIDEOS, GRID_SIZE, BINS, n=10)

processing 10/10

In [504]:
# Print statistics
print("TRAINING SET:")
print("Num. videos:    {:d}".format( len(training_set)) )
print("Num. segments:  {:d}".format( np.sum([len(video.segments) for video in training_set])) )
print("Duration:       {:,.1f} s".format( np.sum([np.sum([segment.duration() for segment in video.segments]) for video in training_set])) )
print("Num frames:     {:d}".format( np.sum([np.sum([segment.num_frames() for segment in video.segments]) for video in training_set])) )
print("Num histograms: {:d}".format( np.sum([np.sum([len(segment.histograms) for segment in video.segments]) for video in training_set])) )

print("\nTEST SET:")
print("Size: {:d}".format( len(test_set) ))

TRAINING SET:
Num. videos:    2
Num. segments:  189
Duration:       529.7 s
Num frames:     14731
Num histograms: 14731

TEST SET:
Size: 10


# Small manual test

In [505]:
for i, test_segment in enumerate(test_set):
    found = search.knownImageSearch(test_segment, training_set, cv2.HISTCMP_CHISQR_ALT, 5, HIST_FRAME_SKIP)
    
    print('Found {} - Expected {}'.format(found, labels[i]))

Found ('00001.mp4', 3351.0) - Expected ('00001.mp4', 3168, 3668)
Found ('00002.mp4', 1690.5) - Expected ('00002.mp4', 1465, 2064)
Found ('00001.mp4', 1887.5) - Expected ('00001.mp4', 1601, 2101)
Found ('00001.mp4', 539.5) - Expected ('00001.mp4', 326, 826)
Found ('00001.mp4', 6064.0) - Expected ('00001.mp4', 5770, 6270)
Found ('00001.mp4', 2804.0) - Expected ('00001.mp4', 2472, 2972)
Found ('00001.mp4', 4857.0) - Expected ('00001.mp4', 4615, 5115)
Found ('00002.mp4', 2991.0) - Expected ('00002.mp4', 2683, 3282)
Found ('00002.mp4', 4573.5) - Expected ('00002.mp4', 4351, 4950)
Found ('00001.mp4', 5310.5) - Expected ('00001.mp4', 5008, 5508)


## Run model on test set

In [172]:
for method in [cv2.HISTCMP_CORREL, cv2.HISTCMP_CHISQR, cv2.HISTCMP_INTERSECT,
               cv2.HISTCMP_BHATTACHARYYA, cv2.HISTCMP_CHISQR_ALT, cv2.HISTCMP_KL_DIV]:
    %timeit -n 1 search.knownImageSearch(test_set[0], training_set, cv2.HISTCMP_CHISQR_ALT, 5, HIST_FRAME_SKIP)   

92.9 ms ± 698 µs per loop (mean ± std. dev. of 7 runs, 5 loops each)
93.6 ms ± 2.06 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)
92.6 ms ± 983 µs per loop (mean ± std. dev. of 7 runs, 5 loops each)
93.1 ms ± 1.88 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)
95.1 ms ± 1.67 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)
97.2 ms ± 6.8 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)


In [496]:
results = []

for i, test_segment in enumerate(test_set):
    print("\rSearching segment {}/{}".format(i+1, len(test_set), len(test_segment)), end='', flush=True)
    
    results.append(search.knownImageSearch(test_segment, training_set, cv2.HISTCMP_CHISQR_ALT, 5, 
                                    HIST_FRAME_SKIP))


Searching segment 10/10

## Evaluate performance

In [497]:
movie_results, start_frame_dist = evaluate(results, labels)

fractions = (movie_results[0] / movie_results[2]*100 if movie_results[2] > 0 else 0, \
             movie_results[1] / movie_results[0]*100 if movie_results[0] > 0 else 0)

print("TEST RESULTS\n")

printParams()
print("\nCorrect video: {:d} / {:d} ({:.1f}%)".format(movie_results[0], movie_results[2], fractions[0]))
print("Inside fragment: {:d} / {:d} ({:.1f}%)".format(movie_results[1], movie_results[0], fractions[1]))
print("Average distance to center of segment: {:.0f} +/- {:.0f} frames (approx. {:.1f} sec)".format(
    start_frame_dist[0], start_frame_dist[1], start_frame_dist[0]/30))

TEST RESULTS

Num. Vid 2 - Grid 2 - Bins [45, 64] - Skip 2

Correct video: 10 / 10 (100.0%)
Inside fragment: 10 / 10 (100.0%)
Average distance to center of segment: 80 +/- 54 frames (approx. 2.7 sec)


In [352]:
def gen():
    for i in range(1,20):
        yield(i)
        
g = gen()

for i in g:
    print(i)
    for _ in range(2):
        try:
            print('inner', next(g))
        except StopIteration:
            pass
    

1
inner 2
inner 3
4
inner 5
inner 6
7
inner 8
inner 9
10
inner 11
inner 12
13
inner 14
inner 15
16
inner 17
inner 18
19
